In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm_notebook as tqdm
import sys
sys.path.append("../src/")
from logger import setup_logger, LOGGER
from trainer import train_lgbm
from util_tool import reduce_mem_usage
%matplotlib inline
pd.set_option('display.max_columns', 300)

In [2]:
# ==================
# Constant
# ==================
TRAIN_PATH = "../input/train.csv"
TEST_PATH = "../input/test.csv"
USER_PATH = "../input/user_x_anime.csv"

In [3]:
# =====================
# Settings
# =====================
SAVE_PATH = "../output/fe/fe020.feather"

In [10]:
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)
user = pd.read_csv(USER_PATH)

In [11]:
train

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,Producers,Licensors,Studios,Source,Duration,Rating,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,Original,24 min. per ep.,R - 17+ (violence & profanity),1251960,61971,105808,718161,71513,26678,329800
1,16,Hachimitsu to Clover,8.06,"Comedy, Drama, Josei, Romance, Slice of Life",Honey and Clover,ハチミツとクローバー,TV,24,"Apr 15, 2005 to Sep 27, 2005",Spring 2005,"Genco, Fuji TV, Shueisha","VIZ Media, Discotek Media",J.C.Staff,Manga,23 min. per ep.,PG-13 - Teens 13 or older,214499,4101,11909,81145,11901,11026,98518
2,22,Tennis no Ouji-sama,7.90,"Action, Comedy, Sports, School, Shounen",The Prince of Tennis,テニスの王子様,TV,178,"Oct 10, 2001 to Mar 23, 2005",Fall 2001,"Production I.G, Nihon Ad Systems",VIZ Media,Trans Arts,Manga,22 min. per ep.,PG-13 - Teens 13 or older,141832,3124,11235,76881,12905,12516,28295
3,23,Ring ni Kakero 1,6.38,"Action, Shounen, Sports",Unknown,リングにかけろ１,TV,12,"Oct 6, 2004 to Dec 15, 2004",Fall 2004,Unknown,Unknown,Toei Animation,Manga,25 min. per ep.,PG - Children,3648,16,170,1333,145,296,1704
4,47,Akira,8.17,"Action, Military, Sci-Fi, Adventure, Horror, S...",AKIRA,AKIRA（アキラ）,Movie,1,"Jul 16, 1988",Unknown,"Mainichi Broadcasting System, Kodansha, Toho","Funimation, Bandai Entertainment, Geneon Enter...",Tokyo Movie Shinsha,Manga,2 hr. 4 min.,R+ - Mild Nudity,566538,9555,8258,434086,3491,2174,118529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,44407,Shinkai,7.23,Music,Unknown,心海,Music,1,"Dec 7, 2020",Unknown,Unknown,Unknown,Unknown,Original,4 min.,G - All Ages,437,6,6,396,1,2,32
4996,45753,Tensei shitara Slime Datta Ken: Kanwa - Hinata...,6.54,"Fantasy, Shounen",That Time I Got Reincarnated as a Slime Season...,転生したらスライムだった件 閑話:ヒナタ・サカグチ,Special,1,"Jan 5, 2021",Unknown,Unknown,Unknown,8bit,Light novel,23 min.,PG-13 - Teens 13 or older,18717,49,1436,12325,172,120,4664
4997,45782,D4DJ Petit Mix,6.51,"Music, Comedy",D4DJ Petit Mix,ぷっちみく♪,TV,Unknown,"Feb 5, 2021 to ?",Winter 2021,Unknown,Unknown,"W-Toon Studio, DMM.futureworks",Original,3 min.,PG-13 - Teens 13 or older,1575,8,908,1,18,22,626
4998,45999,Enen no Shouboutai Mini Anime,5.92,Comedy,Unknown,炎炎ノ消防隊 ミニアニメ,ONA,Unknown,"Jan 8, 2021 to ?",Unknown,Unknown,Unknown,Unknown,Manga,1 min.,Unknown,5392,19,1053,1,178,109,4051


In [13]:
train = pd.concat([train[["MAL_ID","Episodes"]],test[["MAL_ID","Episodes"]]]).reset_index(drop=True)

In [14]:
train.loc[train["Episodes"] == "Unknown","Episodes"] = np.nan
train["Episodes"] = train["Episodes"].astype(float)

In [15]:
user.dtypes

user_id               int64
anime_id              int64
rating              float64
watching_status       int64
watched_episodes      int64
dtype: object

In [16]:
user = user.merge(train,how="left",left_on = "anime_id",right_on = "MAL_ID")

In [18]:
user["episode_rate"] = user["watched_episodes"] / user["Episodes"]

In [19]:
# aggregation
# count
user_rate = user.groupby(by="anime_id")["episode_rate"].mean().to_dict()
user_mean = user.groupby(by="anime_id")["watched_episodes"].mean().to_dict()
#user_unique_count = user[["anime_id","user_id"]].groupby(by="anime_id")["user_id"].nunique().to_dict()

In [20]:
train["anime_id_episode_rate"] = train["MAL_ID"].map(user_rate )
train["anime_id_episode_mean"] = train["MAL_ID"].map(user_mean )

In [21]:
train = reduce_mem_usage(train)
train.iloc[:,-2:].to_feather(SAVE_PATH)

Memory usage of dataframe is 0.38 MB
column =  4
0
Memory usage after optimization is: 0.19 MB
Decreased by 50.0%


In [22]:
train.iloc[:,-2:]

,anime_id_episode_rate,anime_id_episode_mean
0,0.674418,17.534880
1,0.424458,10.186996
2,0.553845,98.584480
3,0.380522,4.566265
4,2.082155,2.082155
...,...,...
12416,0.940000,0.940000
12417,0.457065,1.828258
12418,0.888889,0.888889
12419,0.676214,0.676214
